In [1]:
# Some imports
import sys
import os, glob
from mfx.processlocalizations import ProcessLocalizations
import re
from matplotlib import pyplot as plt
import math
import numpy as np
from mfx import mfxcolnames as col

In [2]:
# Get file names

OUTDIR_LOC = 'C:/Users/apoliti/Desktop/mflux_zarr_tmp_storage/analysis/' # Main directory to store zarr files
OUTDIR_REM =  'Z:/siva_minflux/analysis/'  # Main directory to store results 
INDIR  = 'Z:/siva_minflux/data/'       # main directory of msr file

# Multiple washes with different imager strand
indir_mwash = {'Syp_ATG9': INDIR + '/Multiwash/Syp_ATG9/', 
             'ZnT3_Syp': INDIR + '/Multiwash/ZnT3_Syp/'}
outdir_mwash = {'Syp_ATG9': OUTDIR_REM +  '/Multiwash/Syp_ATG9/', 
                'ZnT3_Syp': OUTDIR_REM +  '/Multiwash/ZnT3_Syp/'}
zarrdir_mwash = {'Syp_ATG9': OUTDIR_LOC + '/Multiwash/Syp_ATG9', 
                'ZnT3_Syp': OUTDIR_LOC + '/Multiwash/ZnT3_Syp/'}

# Wash with a single imager strand
indir_swash = {'Syp': INDIR + '/Single wash/Syp/', 
             'ATG9': INDIR + '/Single wash/ATG9/'}
outdir_swash = {'Syp': OUTDIR_REM +  '/Single wash/Syp/', 
                'ATG9': OUTDIR_REM +  '/Single wash/ATG9/'}
zarrdir_swash = {'Syp': OUTDIR_LOC +  '/Single wash/Syp/', 
                'ATG9': OUTDIR_LOC +  '/Single wash/ATG9/'}


# Consitency controls. Wash with a single imager strand but multiple times. 
indir_cwash = {'VGLUT1_VGLUT1': INDIR + '/Multiwash/VGLUT1_VGLUT1/'}
outdir_cwash = {'VGLUT1_VGLUT1': OUTDIR_REM +  '/Multiwash/VGLUT1_VGLUT1/'}
zarrdir_cwash = {'VGLUT1_VGLUT1': OUTDIR_LOC +  '/Multiwash/VGLUT1_VGLUT1/'}



npy_mwash = {'Syp_ATG9': [], 'ZnT3_Syp': []}

for key in outdir_mwash:
    for (root, dirs, files) in os.walk(outdir_mwash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_mwash[key].append(npyfile)
                    

npy_swash = {'Syp': [], 'ATG9': []}

for key in outdir_swash:
    for (root, dirs, files) in os.walk(outdir_swash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_swash[key].append(npyfile)

npy_cwash = {'VGLUT1_VGLUT1': []}

for key in outdir_cwash:
    for (root, dirs, files) in os.walk(outdir_cwash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_cwash[key].append(npyfile)

pl = ProcessLocalizations(npy_cwash['VGLUT1_VGLUT1'][1])
pl.log_parameters()

2022-09-21 22:29:10,845 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220811_VGLUT1_ROI01_First\220811_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-08
DBCLUSTER_EPS_ALL 2e-08
DBCLUSTER_EPS_MERGED_ALL 2e-08
DBCLUSTER_EPS_MERGED_MEAS 2e-08
logger <Logger processlocalizations (INFO)>



In [4]:
def processFile(file_name, eps_range):
    pl = ProcessLocalizations(file_name)
    pl.trim_min_localizations()
    # Split tracks if needed, remove outliers in each track
    pl.cluster_tid(method=pl.CLS_METHOD_BAYES_GMM)

    # find objects that belong to the same cluster according to distance 30 nm
    for eps in eps_range:
            
        pl.DBCLUSTER_EPS_MEAS = eps
        pl.DBCLUSTER_EPS_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_ALL = eps
        pl.DBCLUSTER_EPS_MERGED_MEAS = eps
        pl.log_parameters()
    
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
        pl.cluster_all_intersect(col.CLS_ALL)
        summary_dict = pl.summary_per_tid2()
        eps_txt = '%d' % (eps*1e9)
        pl.export_csv(summary_dict, file_path=pl.file_path_no_ext() + '_eps' + eps_txt)
        pl.export_vtu(in_dict=summary_dict, coord=col.LTR, file_path=pl.file_path_no_ext() + '_eps' +  eps_txt)
    

In [6]:
eps_range = [5e-9, 1e-8, 2e-8, 3e-8, 4e-8, 5e-8, 1e-7]
processFile(npy_swash['Syp'][0], eps_range=[2e-9])

2022-09-21 22:31:41,290 [INFO] **** trim_min_localizations ****
220825_Syp_P1 Removed 281/1354 tracks with less than 3 localizations

2022-09-21 22:31:53,523 [INFO] **** cluster_tid ****
220825_Syp_P1 MIN_SPLIT_LOCALIZATION: 6, sd_limit: 10.03 nm
Processed TID: 228 / 1073, Total tracks TID2: 1208

2022-09-21 22:31:53,525 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Single wash/Syp/220825_Syp_ROI2\220825_Syp_ROI2.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-09
DBCLUSTER_EPS_ALL 2e-09
DBCLUSTER_EPS_MERGED_ALL 2e-09
DBCLUSTER_EPS_MERGED_MEAS 2e-09
logger <Logger processlocalizations (INFO)>

2022-09-21 22:31:53,724 [INFO] **** cluster_meas ****
220825_Syp_P1, Method dbscan, eps: 2.00 nm
Total tracks TID2: 1207, total clusters meas: 1207

2022-09-21 22:31:53,734 [INFO] **** cluster_all ****
*******cluster_all*******
Only one wash. Nothing to do
2022-09-21 22:31:53,736 [INFO] **** cluster_all_inter

In [1]:
sum(pl.loc['220825_Syp_P1'][col.TID2] == -1)

NameError: name 'pl' is not defined

In [15]:
eps_txt = '%d' % (eps*1e9)